In [ ]:
import pandas as pd
import seaborn as sns


In [ ]:
import os

print(os.getcwd())

In [ ]:
# issue urls where we couldn't parse llm response
file_path = '/path/to/input/files' % replace with real path
removed_issue_urls_df = pd.read_csv(file_path)
removed_issue_urls=removed_issue_urls_df['issue_url'].tolist()
removed_issue_urls

In [ ]:
len(removed_issue_urls)

In [ ]:
file_path = '/path/to/input/files' % replace with real path

df_pred_llama = pd.read_csv(file_path)
df_pred_llama = df_pred_llama[~df_pred_llama['issue_url'].isin(removed_issue_urls)]
df_pred_llama

In [ ]:
file_path = '/path/to/input/files' % replace with real path

df_pred_qwen = pd.read_csv(file_path)
df_pred_qwen= df_pred_qwen.rename(columns={'issue_id':'issue_url'})

df_pred_qwen = df_pred_qwen[~df_pred_qwen['issue_url'].isin(removed_issue_urls)]
df_pred_qwen

## LLM Prediction Stats

In [ ]:
file_path = '/path/to/input/files' % replace with real path
train_issue_ids = pd.read_csv(file_path)['issue_id'].tolist()

file_path = '/path/to/input/files' % replace with real path
test_issue_ids = pd.read_csv(file_path)['issue_id'].tolist()
train_issue_ids=[str(x) for x in train_issue_ids]
test_issue_ids=[str(x) for x in test_issue_ids]

In [ ]:
file_path = '/path/to/input/files' % replace with real path

df_unified_pred_llama = pd.read_csv(file_path)
df_unified_pred_llama['issue_id']=df_unified_pred_llama['issue_id'].astype(str)
df_train_pred_llama = df_unified_pred_llama[df_unified_pred_llama['issue_id'].isin(train_issue_ids)]
df_test_pred_llama = df_unified_pred_llama[~df_unified_pred_llama['issue_id'].isin(train_issue_ids)]
df_test_pred_llama

In [ ]:
file_path = '/path/to/input/files' % replace with real path

df_unified_pred_qwen = pd.read_csv(file_path)
df_unified_pred_qwen['issue_id']=df_unified_pred_qwen['issue_id'].astype(str)
df_train_pred_qwen = df_unified_pred_qwen[df_unified_pred_qwen['issue_id'].isin(train_issue_ids)]
df_test_pred_qwen = df_unified_pred_qwen[~df_unified_pred_qwen['issue_id'].isin(train_issue_ids)]
df_test_pred_qwen

In [ ]:
print("Llama toxicity score statistics on train set:")
print(df_train_pred_llama['toxicity_score'].describe())

print()
print("Llama toxicity score statistics on test set:")
print(df_test_pred_llama['toxicity_score'].describe())

print()
print("Qwen toxicity score statistics on train set:")
print(df_train_pred_qwen['toxicity_score'].describe())

print()
print("Qwen toxicity score statistics on test set:")
print(df_test_pred_qwen['toxicity_score'].describe())

In [ ]:
file_path = '/path/to/input/files' % replace with real path

df_comments = pd.read_csv(file_path)
df_comments = df_comments[~df_comments['issue_url'].isin(removed_issue_urls)]
df_comments

In [ ]:
df_comments.columns.tolist()

In [ ]:
file_path = '/path/to/input/files' % replace with real path

df_conv = pd.read_csv(file_path)
df_conv = df_conv[~df_conv['issue_url'].isin(removed_issue_urls)]
df_conv

In [ ]:
df_conv.columns.tolist()

In [ ]:
# These are the correct verified converasations by the verifier
file_path = '/path/to/input/files' % replace with real path

df_incorrect = pd.read_csv(file_path)

df_incorrect

In [ ]:
print(f"Llama predicted toxic {sum(df_incorrect['toxicity_score_llama']>=0.3)}")
print(f"Qwen predicted toxic {sum(df_incorrect['toxicity_score_qwen']>=0.3)}")

In [ ]:
print(f"Llama predicted toxic {df_incorrect['is_toxic_llm_pred_llama'].sum()}")
print(f"Qwen predicted toxic {df_incorrect['is_toxic_llm_pred_qwen'].sum()}")

In [ ]:
# These are the correct verified converasations by the verifier
file_path = '/path/to/input/files' % replace with real path

df_correct = pd.read_csv(file_path)
df_correct = df_correct.rename(columns= {
    'is_toxic_llm_pred_llama':'is_toxic'
})
df_correct

In [ ]:
file_path = '/path/to/input/files' % replace with real path

df_manual_annotated = pd.read_csv(file_path)
df_manual_annotated

In [ ]:
df_labelled_combined = pd.concat([df_correct[['issue_url', 'is_toxic']], df_manual_annotated], axis=0, ignore_index=True)
df_labelled_combined

In [ ]:
df_labelled_combined['is_toxic'].value_counts()

In [ ]:
df_conv.shape

In [ ]:
df_conv_merged = df_conv.merge(df_labelled_combined, on='issue_url', how='inner')
df_conv_merged

In [ ]:
toxic_df=df_conv_merged[df_conv_merged['is_toxic']==1]
toxic_df

In [ ]:
toxic_urls = toxic_df['issue_url'].tolist()
len(toxic_urls)

In [ ]:
# df_comments = df_comments[df_comments['issue_url'].isin(correct_urls)].reset_index(drop=True)
# del df_comments
# df_comments

In [ ]:
df_toxic_conv_comments = df_comments[df_comments['issue_url'].isin(toxic_urls)]

## Hypothesis #2 (Saveski et al): We find a clear positive relationship between size and toxicity. Larger trees tend to be more toxic in both datasets.

In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import re
from collections import defaultdict
from urllib.parse import urlparse

def extract_mentioned_users(text):
    """Extract all users mentioned with @username in the comment text."""
    if not isinstance(text, str):
        return []
    # Pattern to match @username (alphanumeric characters and hyphens)
    pattern = r'@([a-zA-Z0-9-]+)'
    return re.findall(pattern, text)

def create_mention_graphs(df):
    """Create a directed graph for each issue based on user mentions."""
    # Group comments by issue_url
    issue_groups = df.groupby('issue_url')
    
    # Store graphs for each issue along with the issue_url
    issue_graphs = {}
    
    for issue_url, group in issue_groups:
        # Create a directed graph for this issue
        G = nx.DiGraph()
        
        # Add all users from this issue as nodes (even if they don't mention anyone)
        for user in group['user'].unique():
            G.add_node(user)
        
        # Process each comment to add edges for mentions
        for _, row in group.iterrows():
            commenter = row['user']
            mentioned_users = extract_mentioned_users(row['body'])
            
            # Add edges from commenter to each mentioned user
            for mentioned in mentioned_users:
                # Only add edge if the mentioned user exists in our dataset for this issue
                if mentioned in group['user'].values:
                    G.add_edge(commenter, mentioned)
        
        # Store the graph with the issue_url as key
        issue_graphs[issue_url] = G
    
    return issue_graphs

def calculate_deepest_paths(issue_graphs):
    """Calculate the length of the longest path in each graph."""
    path_lengths = {}
    
    for issue_url, G in issue_graphs.items():
        # Find the longest path length in the graph
        if len(G.nodes()) <= 1:  # No path if 0 or 1 node
            path_lengths[issue_url] = 0
        else:
            # Check if the graph is a DAG (Directed Acyclic Graph)
            try:
                # For DAGs, we can find the longest path
                longest_path_length = nx.dag_longest_path_length(G)
                path_lengths[issue_url] = longest_path_length
            except nx.NetworkXUnfeasible:
                # If there are cycles, use the more general approach
                # Find all simple paths between all pairs of nodes and get the longest
                all_path_lengths = []
                
                for source in G.nodes():
                    for target in G.nodes():
                        if source != target:
                            try:
                                # Get all simple paths from source to target
                                paths = list(nx.all_simple_paths(G, source, target))
                                if paths:
                                    # Find the longest path length
                                    max_path_length = max(len(path) - 1 for path in paths)
                                    all_path_lengths.append(max_path_length)
                            except nx.NetworkXNoPath:
                                continue
                
                path_lengths[issue_url] = max(all_path_lengths) if all_path_lengths else 0
    
    return path_lengths

def create_path_length_dataframe(issue_graphs):
    """Create a dataframe with issue_url and deepest path length."""
    path_lengths = calculate_deepest_paths(issue_graphs)
    
    # Create dataframe
    df_path_lengths = pd.DataFrame({
        'issue_url': list(path_lengths.keys()),
        'deepest_path_length': list(path_lengths.values())
    })
    
    return df_path_lengths


In [ ]:
# Create graphs for each issue
issue_graphs = create_mention_graphs(df_comments)

# Create dataframe with issue_url and deepest path length
df_path_lengths = create_path_length_dataframe(issue_graphs)

# Print the result
print("Dataframe with issue URLs and deepest path lengths:")
df_path_lengths

In [ ]:
df_path_lengths['deepest_path_length'].describe()

In [ ]:
df_path_lengths[df_path_lengths['deepest_path_length']>=6]['issue_url'].to_list()

In [ ]:
df_test = df_conv_merged[df_conv_merged['issue_url']=='https://api.github.com/repos/flutter/flutter/issues/154241']
df_test.to_csv('7_mention_conv.csv', index=False)

In [ ]:
df_test['is_toxic']

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def analyze_toxicity_by_path_length(df_path_lengths, df_correct_filtered):
    """
    Analyze the percentage of toxic issues for each deepest_path_length category.
    
    Parameters:
    df_path_lengths: DataFrame with issue_url and deepest_path_length
    df_correct_filtered: DataFrame with issue_url and is_toxic
    
    Returns:
    DataFrame with analysis results
    """
    # Merge the two dataframes on issue_url
    merged_df = pd.merge(df_path_lengths, df_correct_filtered, on='issue_url', how='inner')
    
    # Calculate the total number of toxic issues
    total_toxic_issues = merged_df[merged_df['is_toxic'] == True].shape[0]
    
    # Group by deepest_path_length and calculate toxicity metrics
    toxicity_by_path = merged_df.groupby('deepest_path_length').agg(
        total_issues=('issue_url', 'count'),
        toxic_issues=('is_toxic', lambda x: sum(x == True)),
        non_toxic_issues=('is_toxic', lambda x: sum(x == False))
    ).reset_index()
    
    # Calculate percentages
    toxicity_by_path['toxic_percentage_within_category'] = (toxicity_by_path['toxic_issues'] / 
                                                           toxicity_by_path['total_issues'] * 100).round(2)
    
    toxicity_by_path['percentage_of_all_toxic_issues'] = (toxicity_by_path['toxic_issues'] / 
                                                         total_toxic_issues * 100).round(2)
    
    return toxicity_by_path

def visualize_toxicity_distribution(toxicity_by_path):
    """Create visualizations for the toxicity distribution by path length."""
    # Set up the figure with two subplots
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 12))
    
    # Plot 1: Percentage of toxic issues within each path length category
    sns.barplot(x='deepest_path_length', y='toxic_percentage_within_category', 
                data=toxicity_by_path, ax=ax1, palette='Blues_d')
    # ax1.set_title('Percentage of Toxic Issues Within Each Path Length Category')
    ax1.set_xlabel('Deepest Path Length')
    ax1.set_ylabel('Percentage of Issues That Are Toxic')
    # Add "(a)" label at the bottom-middle of the first subplot
    ax1.text(0.5, -0.18, "(a)", transform=ax1.transAxes, 
             ha='center', va='center', fontsize=16)
    
    # Add value labels
    for i, row in enumerate(toxicity_by_path.itertuples()):
        ax1.text(i, row.toxic_percentage_within_category + 1, 
                f"{row.toxic_percentage_within_category}%", 
                ha='center')
    
    # Plot 2: Distribution of all toxic issues by path length
    sns.barplot(x='deepest_path_length', y='percentage_of_all_toxic_issues', 
                data=toxicity_by_path, ax=ax2, palette='Reds_d')
    # ax2.set_title('Distribution of All Toxic Issues by Path Length')
    ax2.set_xlabel('Deepest Path Length')
    ax2.set_ylabel('Percentage of All Toxic Issues')
      # Add "(a)" label at the bottom-middle of the first subplot
    ax2.text(0.5, -0.18, "(b)", transform=ax2.transAxes, 
             ha='center', va='center', fontsize=16)
    
    # Add value labels
    for i, row in enumerate(toxicity_by_path.itertuples()):
        ax2.text(i, row.percentage_of_all_toxic_issues + 1, 
                f"{row.percentage_of_all_toxic_issues}%", 
                ha='center')
    
    plt.tight_layout()
    plt.savefig('./output/toxicity_by_path_length.png', dpi=1200, bbox_inches='tight')
    plt.show()
    
    return fig

# Main function
def analyze_toxicity_path_relationship(df_path_lengths, df_correct_filtered):
    """Main function to analyze and visualize the relationship between path length and toxicity."""
    # Perform the analysis
    toxicity_analysis = analyze_toxicity_by_path_length(df_path_lengths, df_correct_filtered)
    
    # Print the results table
    print("Analysis of Toxicity by Deepest Path Length:")
    print(toxicity_analysis)
    print("\nInterpretation:")
    print(f"- Total toxic issues analyzed: {toxicity_analysis['toxic_issues'].sum()}")
    
    # For each path length, print the percentage of all toxic issues
    for _, row in toxicity_analysis.iterrows():
        print(f"- {row['percentage_of_all_toxic_issues']}% of the overall total toxic issues have deepest_path_length {row['deepest_path_length']}")
    
    # Create visualizations
    visualize_toxicity_distribution(toxicity_analysis)
    
    return toxicity_analysis

# Example usage
if __name__ == "__main__":
    # Assuming df_path_lengths and df_correct_filtered are already defined
    
    toxicity_analysis = analyze_toxicity_path_relationship(df_path_lengths, df_conv_merged)

In [ ]:
"""Main function to analyze and visualize the relationship between path length and toxicity."""
# Perform the analysis
toxicity_analysis = analyze_toxicity_by_path_length(df_path_lengths, df_conv_merged)

# Print the results table
print("Analysis of Toxicity by Deepest Path Length:")
print(toxicity_analysis)
print("\nInterpretation:")
print(f"- Total toxic issues analyzed: {toxicity_analysis['toxic_issues'].sum()}")

# For each path length, print the percentage of all toxic issues
for _, row in toxicity_analysis.iterrows():
    print(f"- {row['percentage_of_all_toxic_issues']}% of the overall total toxic issues have deepest_path_length {row['deepest_path_length']}")


## Hypothesis #10(Ferreira et al.): Use of capital letters in uncivil threads
**Quote:** “Contributors tend to express Annoyance and Bitter frustration in issue discussions when they use capital letters to emphasize something in a frustrating way, when someone is using abusive language to express their opinion, when injustice makes the other person feel unable to defend herself/himself, and when the speaker is strongly irritated by something impossible to do in the speaker’s opinion.”

In [ ]:
import pandas as pd
import re

def analyze_capital_letters(df):
    """
    Analyzes a dataframe of comments to calculate:
    1) Total number of all-capital-letter comments in each conversation
    2) Total number of comments where at least one word is all capital letters
    
    Parameters:
    df (pandas.DataFrame): DataFrame with columns 'issue_url' and 'body'
    
    Returns:
    pandas.DataFrame: Results dataframe with capital letter metrics by issue
    """
    # Function to check if a comment is entirely in capital letters
    def is_all_caps(text):
        # Check if text contains any letters
        if not re.search('[a-zA-Z]', text):
            return False
        # Check if all letters are uppercase
        return all(c.isupper() for c in text if c.isalpha())
    
    # Function to check if any word in a comment is entirely in capital letters
    def has_caps_word(text):
        words = re.findall(r'\b[a-zA-Z]+\b', text)
        # Check if any word has at least four letters
        # to remove single letter word like "A big cat"
        # and to remove abbreviations here like, AI, TCP, MCP, ...             
        # and all letters are uppercase
        return any(len(word) >= 4 and all(c.isupper() for c in word) for word in words)
    
    # Create temporary columns with our calculated values
    df['is_all_caps'] = df['body'].apply(is_all_caps)
    df['has_caps_word'] = df['body'].apply(has_caps_word)
    
    # Group by issue_url and count the occurrences
    results = df.groupby('issue_url').agg(
        total_comments=('body', 'count'),
        all_caps_comments=('is_all_caps', 'sum'),
        comments_with_caps_word=('has_caps_word', 'sum')
    ).reset_index()
    
    # Calculate percentages
    results['all_caps_percentage'] = (results['all_caps_comments'] / results['total_comments'] * 100).round(2)
    results['caps_word_percentage'] = (results['comments_with_caps_word'] / results['total_comments'] * 100).round(2)
    
    return results

# Example usage:
results_df = analyze_capital_letters(df_comments)
results_df

In [ ]:
toxic_urls = df_conv_merged[df_conv_merged['is_toxic']==True ]['issue_url'].tolist()
df_toxic_cap = results_df[results_df['issue_url'].isin(toxic_urls)]

print("Analysis of the percentage of comments containig at least one all capital letter word in the toxic conversations")
print(df_toxic_cap['caps_word_percentage'].describe())

non_toxic_urls = df_conv_merged[df_conv_merged['is_toxic']==False ]['issue_url'].tolist()
df_non_toxic_cap = results_df[results_df['issue_url'].isin(non_toxic_urls)]
print("Analysis of the percentage of comments containing at least one all capital letter word in the non-toxic conversations")
print(df_non_toxic_cap['caps_word_percentage'].describe())

## Hypothesis (Sarkar et al): Accepted PRs are less likely to encounter toxicity

In [ ]:
df_issue_acceptance = pd.read_csv("./output/issue_acceptance_data_large_dataset.csv")
df_issue_acceptance

In [ ]:
total_issues = df_issue_acceptance[df_issue_acceptance['is_pull_req']==True].shape[0]
total_issues

In [ ]:
# remove the invalid issues
df_issue_acceptance = df_issue_acceptance[~df_issue_acceptance['issue_url'].isin(removed_issue_urls)]
df_issue_acceptance

In [ ]:
df_pr_status = df_issue_acceptance[df_issue_acceptance['is_pull_req']==True]
del df_issue_acceptance
df_pr_status

In [ ]:
# Only take the pr issues that are in our labelled data
df_pr_status = df_pr_status[df_pr_status['issue_url'].isin(df_conv_merged['issue_url'].tolist())]
df_pr_status

In [ ]:
df_conv_merged['is_toxic'].unique()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Assuming df_pr_status and df_conv_merged are already loaded
# Merge the dataframes based on issue_url
merged_df = pd.merge(df_pr_status, df_conv_merged, on='issue_url', how='left')
print(f"Total pull request(PR) in our dataset {merged_df.shape[0]}" )
print("Among these,")

accepted_df =  merged_df[ merged_df['is_merged']==True]
accepted_toxic_df = accepted_df[accepted_df['is_toxic']==True]
print("\nFrom accepted PR perspective,")
print(f'\tOut of {accepted_df.shape[0]} accepted pr, {accepted_toxic_df.shape[0]} are toxic. Toxicity rate ={accepted_toxic_df.shape[0]/accepted_df.shape[0]*100:.2f}%')

non_accepted_df =  merged_df[merged_df['is_merged']==False]
non_accepted_toxic_df = non_accepted_df[non_accepted_df['is_toxic']==True]
print(f'\tOut of {non_accepted_df.shape[0]} non-accepted pr, {non_accepted_toxic_df.shape[0]} are toxic. Toxicity rate ={non_accepted_toxic_df.shape[0]/non_accepted_df.shape[0]*100:.2f}%')

toxic_df =  merged_df[ merged_df['is_toxic']==True]
toxic_accepted_df = toxic_df[toxic_df['is_merged']==True]
print("\nToxic PR perspective,")
print(f'\tOut of {toxic_df.shape[0]} toxic pr, {toxic_accepted_df.shape[0]} are accepted. Acceptance rate ={toxic_accepted_df.shape[0]/toxic_df.shape[0]*100:.2f}%')

nontoxic_df =  merged_df[merged_df['is_toxic']==False]
nontoxic_df
nontoxic_accepted_df = nontoxic_df[nontoxic_df['is_merged']==True]
print(f'\tOut of {nontoxic_df.shape[0]} non-toxic pr, {nontoxic_accepted_df.shape[0]} are accepted. Acceptance rate ={nontoxic_accepted_df.shape[0]/nontoxic_df.shape[0]*100:.2f}%')



## Hypothesis (Imran et al.) : Toxic conversations on GitHub tended to be longer, with a median of 11 comments versus 6 in the non-toxic conversations.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Assuming df_conv is your dataframe with GitHub issue conversations
# and toxic_issue_urls is a list of URLs identified as toxic

# Step 1: Create a new column indicating if an issue is toxic

# Step 2: Separate toxic and non-toxic issues
toxic_issues = df_conv_merged[df_conv_merged['is_toxic'] == True]
non_toxic_issues = df_conv_merged[df_conv_merged['is_toxic'] == False]

# Step 3: Calculate summary statistics
toxic_stats = toxic_issues['comments_count'].describe()
non_toxic_stats = non_toxic_issues['comments_count'].describe()

# Print the summary statistics
print("Toxic Issues Statistics:")
print(toxic_stats)
print("\nNon-Toxic Issues Statistics:")
print(non_toxic_stats)

# Step 4: Compare medians
toxic_median = toxic_issues['comments_count'].median()
non_toxic_median = non_toxic_issues['comments_count'].median()

print(f"\nMedian comment count for toxic issues: {toxic_median}")
print(f"Median comment count for non-toxic issues: {non_toxic_median}")
print(f"Difference in medians: {toxic_median - non_toxic_median}")

# Step 5: Statistical test (Mann-Whitney U Test - non-parametric test for comparing medians)
u_stat, p_value = stats.mannwhitneyu(toxic_issues['comments_count'], non_toxic_issues['comments_count'])
print(f"\nMann-Whitney U Test: U={u_stat}, p-value={p_value}")
if p_value < 0.05:
    print("The difference is statistically significant (p < 0.05)")
else:
    print("The difference is not statistically significant (p ≥ 0.05)")

# Step 6: Visualize the distributions
plt.figure(figsize=(12, 6))

# Boxplot
plt.subplot(1, 2, 1)
sns.boxplot(x='is_toxic', y='comments_count', data=df_conv_merged)
plt.title('Comment Count Comparison')
plt.xlabel('Is Toxic')
plt.ylabel('Number of Comments')

# Histogram/KDE
plt.subplot(1, 2, 2)
sns.histplot(toxic_issues['comments_count'], color='red', alpha=0.5, label='Toxic')
sns.histplot(non_toxic_issues['comments_count'], color='blue', alpha=0.5, label='Non-Toxic')
plt.title('Distribution of Comment Counts')
plt.xlabel('Number of Comments')
plt.ylabel('Frequency')
plt.legend()

plt.tight_layout()
plt.show()

# Step 7: Calculate effect size (Cohen's d)
mean_diff = toxic_issues['comments_count'].mean() - non_toxic_issues['comments_count'].mean()
pooled_std = np.sqrt((toxic_issues['comments_count'].var() + non_toxic_issues['comments_count'].var()) / 2)
cohens_d = mean_diff / pooled_std

print(f"\nEffect size (Cohen's d): {cohens_d}")
if abs(cohens_d) < 0.2:
    print("Small effect size")
elif abs(cohens_d) < 0.5:
    print("Medium effect size")
else:
    print("Large effect size")